# Lab: Frequency Sweep

This lab is inspired by [Calibrating Qubits using Qiskit Pulse](https://learn.qiskit.org/course/quantum-hardware-pulses/calibrating-qubits-using-qiskit-pulse#frequencysweep) from the [Qiskit Textbook](https://qiskit.org/textbook/). It's a short lab, but allows more interactivity for the frequency sweep portion of that chapter and helps showcase some of the features of Pulsemaker. Consider this lab a "toy" example that could work well in a high school setting. In practice, you wouldn't actually perform a frequency sweep this way as it would be inefficient.

_NOTE_: If you are new to Pulsemaker, then you might want to run through the [PulseDesigner tutorial](Tutorial_PulseDesigner.ipynb) and the [ScheduleDesigner tutorial](Tutorial_ScheduleDesigner.ipynb) to be more comfortable with this lab.

## Qubit in a Box

Okay, so you saved up enough money to finally buy yourself a single-qubit device for use at home: the _Qutron 1000_ (aka Armonk among those in the know). Your Amazon box arrived this morning on your doorstep, but upon opening the box you were disappointed to read this included note:

> We apologize for the inconvenience, but we've had so much demand that we opted to send out devices without calibration so that we could fulfill orders in as timely a way as possible. You will need to perform a frequency sweep on your device to determine your qubit's actual frequency. Based on our manufacturing process we know that this will be in the range of **4.965** to **4.983 GHz**. Happy hunting!

Good thing you're a tinkerer at heart. Time to dust off some of your old diagnostic tools!

_Instructional note: run each of the code cells below as you continue through this lab._

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'
from pulsemaker import PulseDesigner, ScheduleDesigner
from qiskit.pulse import SetFrequency
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
from time import sleep

### Waveform generator
You head to your garage to look for that waveform generator you know you have...somwhere. Ahh, there it is behind those stacks of old Popular Science magazine that you just can't bring yourself to get rid of. There she is in all her beauty: _PulseDesigner_. But how are we supposed to drive this qubit? Ahh, there's a second note in the shipping box:

> Please have your waveform match the following and drive with an amplitude of **0.05**.  
![waveform](images/labFrequencySweep_01.png)

You recall that you only need to adjust the in-phase portion of the waveform (i.e. no need to modify Quadrature). So, you first set the amplitude and then continue on to adjust the sample count, so you have a reasonable amount of samples and finally adjust sigma.

In [ ]:
pulse_d = PulseDesigner()
pulse_d

### License to drive

Now that we've got the right waveform we can start driving the qubit and see what response we get. You fire up the included _ScheduleDesigner_ application that came with your Qutron, so that you can create a pulse schedule to send to the device. 

In [ ]:
schedule_d = ScheduleDesigner()
pulse_d.observe(schedule_d.update_custom_pulse, names='pulse')
schedule_d.update_custom_pulse({'new': pulse_d.pulse})

def load():
    out = widgets.Output()
    display(out)
    out.append_stdout('Loading ScheduleDesigner');
    for i in range(3):
        out.append_stdout(f'{"." * i}')
        sleep(1)
    out.append_stdout("Loaded ✅")

GHz = 1.0e9 # Gigahertz    

sweep_values = []
frequencies = []
def plot():
    fig = plt.figure(figsize=(3, 3))
    if len(sweep_values) > 0:
        plt.scatter(frequencies[:-1], np.real(sweep_values[:-1]), color='black') # plot real part of sweep values
        plt.scatter(frequencies[-1], np.real(sweep_values[-1]), color='red') # plot real part of sweep values        
    if len(frequencies) > 1 and min(frequencies) != max(frequencies):
        plt.xlim([min(frequencies), max(frequencies)])
    plt.xlabel("Frequency [GHz]")
    plt.ylabel("Measured signal [a.u.]")
result_plot = widgets.interactive(plot)

def update_results(change):
    result = change['new']
    if result.results is None:
        return
    
    schedule = change.owner.schedule
    for start_time, instruction in schedule.instructions:
        if isinstance(instruction, SetFrequency):
            frequencies.append(instruction.frequency / GHz)
            break
            
    # This scale factor was experimentally determined to give similar results to the "Calibrating Qubits using Qiskit Pulse" chapter, but by only using the simulator
    scale_factor = 36
    for i in range(len(result.results)):
        # Get the results from the ith experiment
        res = 6 - result.get_memory(i) * scale_factor
        # Get the results for `qubit` from this experiment
        sweep_values.append(res[0])

    result_plot.update()
schedule_d.observe(update_results, names='result')

load()

### What's the magic number?

Loaded and ready to go. In order to scan frequencies you know that you need to have a schedule that:
1. Sets a frequency
2. Sends the waveform as a set of pulses

Looking at the instruction manual for _ScheduleDesigner_ you can see that it is possible to do this using the "Freq [GHz]" and "From Pulse" buttons. You hook up your PulseDesigner to your computer, so that you can import the waveform from it. The manual says to turn on **Auto-run**, so that you can see immediate output in the graph on the right. The red dot (🔴) is the last schedule that was run and all black dots (⚫) are previously run schedules. You can modify the frequency by selecting that item in the schedule and clicking 📝 (edit). The manual says that with auto-run you can just enter new values or use the spinner (up/down) buttons and the output should immediately update. It would be good to start with the bounds and then perhaps binary search or you could just go in increments of 0.001 GHz.

What was that range again? Oh yeah, **4.965** to **4.983 GHz**.

Let's search for that signal spike! 

In [ ]:
display(widgets.HBox([schedule_d, result_plot]))

### Let's dial it in

You found that magic number, but it'd be good to be a little more rigorous and fit all of the values you recorded to a curve. You did sweep the full range, right? Right? If your curve does not look like it is fitting well, then go back one step and make sure you have collected enough points.

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats import norm

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

max_sweep = max(sweep_values)
sweep_fit = sweep_values.copy()
sorted_indices = np.argsort(frequencies)
freqs_fit = np.take(frequencies, sorted_indices)
sweep_fit = np.take(sweep_fit, sorted_indices)

fit_params, y_fit = fit_function(freqs_fit,
                                 np.real(sweep_fit), 
                                 lambda x, A, q_freq, B, C: (A / np.pi) * (B / ((x - q_freq)**2 + B**2)) + C,
                                 [-5, 4.97, 1, 5] # initial parameters for curve_fit
                                )

plt.scatter(freqs_fit, np.real(sweep_fit), color='black')
plt.plot(freqs_fit, y_fit, color='red')
plt.xlim([min(freqs_fit), max(freqs_fit)])

plt.xlabel("Frequency [GHz]")
plt.ylabel("Measured Signal [a.u.]")
plt.show()

In [ ]:
A, rough_qubit_frequency, B, C = fit_params
rough_qubit_frequency = rough_qubit_frequency*GHz # make sure qubit freq is in Hz
print(f"After curve-fitting the qubit frequency is estimated to be at {round(rough_qubit_frequency/GHz, 5)} GHz.")

### Verify your results

Now that you've zeroed in on a precise qubit frequency, let's see how you did. Execute the next cell to find out if you figured it out!

In [ ]:
from pulsemaker import get_qiskit_backend
backend = get_qiskit_backend(schedule_d._current_backend)
if np.isclose(backend.defaults().qubit_freq_est[0], rough_qubit_frequency, atol=1e6):
    print("🎉🎉🎉 You have successfully calibrated your Qutron 1000")
else:
    print("🔎 Hmmm... Something is off, so try retracing your steps")